# Détection de doublons dans les rapports de bugs

Projet réalisé en collaboration avec Arthur Mahy(@arthurmahy) et Floriane Ronzon

## Contexte
En raison de la complexité des systèmes logiciels, les bugs logiciels sont répandus. Les entreprises, en particulier les grandes, utilisent généralement des systèmes de suivi des bugs (BTS), également appelés système de suivi des problèmes, pour gérer et suivre les enregistrements des bugs. Outre les développeurs et les testeurs, de nombreux projets, principalement des projets open source, permettent aux utilisateurs de signaler de nouveaux bugs dans leur BTS. Pour ce faire, les utilisateurs doivent remplir un formulaire avec plusieurs champs. Un sous-ensemble important de ces champs fournissent des données catégorielles et n'acceptent que les valeurs qui vont d'une liste fixe d’options (par exemple, composant, version et produit du système). Deux autres champs à remplir importants sont le résumé et la description. Les utilisateurs sont libres d'écrire tout ce qui peut décrire le bug au mieux dans les deux champs et la seule contrainte est le nombre de caractères. La soumission d'un formulaire crée une page, appelée rapport de bug ou rapport de problème, qui contient toutes les informations sur un bug.


## Objectif 
En raison du manque de communication et de synchronisation, les utilisateurs peuvent ne pas savoir qu'un bug spécifique a déjà été soumis et le signaler à nouveau. Identifier les rapports de bugs en double est une tâche importante dans les BTS. Fondamentalement, notre objectif est de développer un système qui prédit si une paire de rapports de bug se rapportent au même bug.

## Import des librairies

In [ ]:
import nltk
from IPython import display
import json
import os
from bs4 import BeautifulSoup
from multiprocessing import Pool, TimeoutError
from time import time
import tqdm
import string
import numpy as np
import math
nltk.download("punkt")

## Analyse des données

Voici un exemple de rapport de bug soumis sur la plateforme. Ces rapports sont stockés sous forme de fichiers HTML dans le dossier 'data'. 

- A : identifiant du bug report
- B : date de création
- C : résumé
- D : produit
- E : composant
- F : l'identifiant du rapport dont le bug report est dupliqué
- G : description

In [ ]:
display.Image("bug-report-eclipse.png")

In [ ]:
# définir le chemin du dossier qui contient les données
FOLDER_PATH = "data/"
PAGE_FOLDER = os.path.join(FOLDER_PATH, 'bug_reports')

# Fichiers d'entraînement et de validation pour les rapports en double
training_file = open(os.path.join(FOLDER_PATH, "training.txt"))
validation_file = open(os.path.join(FOLDER_PATH, "validation.txt"))
word_vec_path = os.path.join(FOLDER_PATH, "glove.42B.300d_clear.txt")

""" 
Permet la lecture des fichiers sur la liste des rapports
"""
def read_dataset(f):
    for line in f:
        line = line.strip()
        
        if len(line) == 0:
            continue
        
        rep1, rep2, label = line.split(',')

        rep1 = int(rep1)
        rep2 = int(rep2)
        label = 1.0 if int(label) > 0 else 0.0 
        
        yield (rep1, rep2, label)
    

training_pairs = list(read_dataset(training_file))
validation_pairs = list(read_dataset(validation_file))

training_reports_set = set()


for report1, report2, _ in training_pairs:
    training_reports_set.add(report1)
    training_reports_set.add(report2)

## Web scraping

On effectue du web scraping sur les pages HTML des rapports de bugs afin d'obtenir les informations des champs remplis par les utilisateurs. Le web scraping est une extraction des informations d'une page web pour les utiliser dans une analyse calculatoire par exemple.

La fonction ```extract_data_from_page``` retourne un dictionnaire avec la structure suivante :

```python
 {"report_id": int, 
  "dup_id": int or None (identifiant du rapport dont il est dupliqué), 
  "component": str, 
  "product": str, 
  "summary": str, 
  "description": str, 
  "creation_date": str} 
```

In [ ]:
def extract_data_from_page(pagepath):
    dic = {}
    
    with open(pagepath) as fp:
        soup = BeautifulSoup(fp, "lxml")

    #Report ID    
    dic["report_id"] = int(soup.title.string.split(' ')[0])
    
    #Duplicate ID
    try:
        dic["dup_id"] = int(soup.find(id="static_bug_status").a.get('href').split('=')[1])
    except:
        dic["dup_id"] = None
    
    #Component
    component = soup.find("td", id="field_container_component")
    dic["component"] = component.text.replace("\n", "").replace("  (show other bugs)", "")

    #Product
    dic["product"] = soup.find("td", id="field_container_product").text.replace("\n", "")

    #Summary
    dic["summary"] = soup.find("span", id="short_desc_nonedit_display").text.replace("\n", "")

    #Description
    dic["description"] = soup.find("pre", class_="bz_comment_text").text
        
    #Creation date
    newtime = soup.find("span", "bz_comment_time").next_element.strip().split(':')
    dic["creation_date"] = newtime[0] + ':' + newtime[1] + ' ' +(newtime[2])[3:]
        
    return dic

### Extraction de texte à partir de HTML

In [ ]:
# Indexer chaque rapport par son identifiant 
index_path = os.path.join(FOLDER_PATH, 'bug_reports.json')

if os.path.isfile(index_path):
    report_index = json.load(open(index_path))
    report_index = {int(report_id): data for report_id,data in report_index.items()}
else:
    # Extraire le contenu d'une page Web 

    files = [os.path.join(PAGE_FOLDER, filename) for filename in os.listdir(PAGE_FOLDER)]
    reports = [extract_data_from_page(f) for f in tqdm.tqdm(files)]
    report_index = dict(((report['report_id'], report) for report in reports ))

    # Enregistrement des données extraites
    json.dump(report_index, open(index_path,'w'))
    

## Prétraitement des données

Le prétraitement des données est une tache cruciale en fouille de données. Cette étape nettoie et transforme les données brutes dans un format qui permet leur analyse, et leur utilisation avec des algorithmes de *machine learning*. En traitement des langages, la *tokenization* et le *stemming* sont des étapes cruciales. Il faut également filtrer les mots sans importance pour l'utilisation des algorithmes.

### Tokenization

Cette étape permet de séparer un texte en séquence de *tokens* (= jetons, ici des mots, symboles ou ponctuation).
Par exemple, la phrase "It's the student's notebook." peut être séparé en liste de tokens de cette manière: ["it", " 's", "the", "student", " 's", "notebook", "."].


In [ ]:
# Ce tokenizer remplace la ponctuation par des espaces et tokenise ensuite les tokens qui sont séparés par des espaces blancs (espace, tabulation, nouvelle ligne).
def tokenize_space_punk(text):
    replace_punctuation = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    text = text.translate(replace_punctuation)
    text = text.lower()
    text = nltk.word_tokenize(text)
    return text

sentence = "It's the student's notebook."
tokens = tokenize_space_punk(sentence)
print(tokens)

### Suppression des stopwords

Certains tokens sont sans importance pour la comparaison, car ils apparaissent dans la majorité des discussions. Les supprimer réduit la dimension du vecteur et accélère les calculs.
Les tokens sans importance pour la comparaison des discussions sont ceux qui reviennent dans tous les types de conversations. On utilise un ensemble de stopwords trouvé sur internet (https://www.ranks.nl/stopwords) avec une liste de plus de 600 english stop words. Les supprimer de nos listes de tokens fait donc gagner du temps de calcul.

In [ ]:
stopwords = json.load(open("data/stopwords.json",'r'))

# Fonction de filtrage des tokens qui retire les stopwords
def filter_tokens(tokens):
    return [word_token for word_token in tokens if not word_token in stopwords]

filtered_tokens = filter_tokens(tokens)
print(filtered_tokens)

### Stemming

La racinisation (stemming) est un procédé de transformation des flexions en leur radical ou racine. Par example, en anglais, la racinisation de "fishing", "fished" and "fish" donne "fish" (stem).
Le stemming permet de gommer les variations morphologiques de mots. Ainsi, on a pu regrouper certains mots qui ont le même sens voire des sens proches (tried et tries par exemple) et diminuer la quantité de vocabulaire utile pour notre modèle.


In [ ]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')

word1 = ['I', 'tried', 'different', 'fishes']

print([stemmer.stem(w) for w in word1])

word2 = ['I', 'will', 'tries', 'only', 'one', 'fishing']
print([stemmer.stem(w) for w in word2])

## Représentation des données


### Bag of Words (BoW)

De nombreux algorithmes demandent des entrées qui sont toutes de la même taille. Cela n'est pas toujours le cas, notamment pour des données textuelles qui peuvent avoir un nombre variable de mots. 

Par exemple, considérons la phrase 1, ”Board games are much better than video games” et la phrase 2, ”Monopoly is an awesome game!” La table ci-dessous montre un exemple d'un moyen de représentation de ces deux phrases en utilisant une représentation fixe : 

|            | an | are | ! | monopoly | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 0 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |


Chaque colonne représente un mot du vocabulaire (de longueur 13), tandis que chaque ligne contient l'occurrence des mots dans une phrase. Ainsi, la valeur 2 à la position (1,7) est due au mot *"games"* qui apparaît deux fois dans la phrase 1. 

Ainsi, chaque ligne étant de longueur 13, on peut les utiliser comme vecteur pour représenter les phrases 1 et 2. Ainsi, c'est cette méthode que l'on appelle *Bag-of-Words* : c'est une représentation de documents par des vecteurs dont la dimension est égale à la taille du vocabulaire, et qui est construite en comptant le nombre d'occurrences de chaque mot. Ainsi, chaque token est ici associé à une dimension.



### TF-IDF


L'utilisation de la fréquence brute d'apparition des mots, comme c'est le cas avec bag-of-words, peut être problématique. En effet, peu de tokens auront une fréquence très élevée dans un document, et de ce fait, le poids de ces mots sera beaucoup plus grand que les autres, ce qui aura tendance à biaiser l'ensemble des poids. De plus, les mots qui apparaissent dans la plupart des documents n'aident pas à les discriminer. Par exemple, le mot "*de*" apparaît dans beaucoup de documents de la base de données, et pour autant, avoir ce mot en commun ne permet pas de conclure que des documents sont similaires. À l’inverse, le mot "*génial*" est plus rare, mais les documents qui contiennent ce mot sont plus susceptibles d'être positifs. TF-IDF est donc une méthode qui permet de pallier à ce problème.

TF-IDF pondère le vecteur en utilisant une fréquence de document inverse (IDF) et une fréquence de termes (TF).

TF est l'information locale sur l'importance qu'a un mot dans un document donné, tandis que IDF mesure la capacité de discrimination des mots dans un jeu de données. 

L'IDF d'un mot se calcule de la façon suivante:
\begin{equation}
	\operatorname{IDF}(t) = \ln\left( \frac{N+1}{\operatorname{df}(t)+1} \right) + 1,
\end{equation}
où $t$ est un token, $N$ est le nombre de documents dans l'ensemble de données, et $\operatorname{df}(\cdot)$  est le nombre de documents qui contiennent un mot $i$.

Le nouveau poids d'un mot $t$ dans un texte peut ensuite être calculé de la façon suivante:

\begin{equation}
	w(t) = \operatorname{tf}(t) \times \operatorname{IDF}(t),
\end{equation}
où $\operatorname{tf}(\cdot)$ est le terme fréquence du mot 𝑖 dans le document 𝑗, c'est-à-dire le nombre de fois qu'un mot apparaît dans le document. *Nous appelons représentation TF-IDF lorsque les poids de la représentation BoW sont calculés au moyen de TF-IDF.*

In [ ]:
class TFIDF:
    idf = {}

    """
    Apprend les valeurs IDF basées sur les données textuelles dans X, la matrice des tokens
    """
    def fit(self, X):
        self.idf.clear()
        for document in X:
            document = set(document)
            for word in document:
                if word in self.idf:
                    self.idf[word] += 1
                else:
                    self.idf[word] = 1
                    
        N = len(X)
        self.idf = {word : np.log((N+1)/(self.idf[word]+1))+1 for word in tqdm.tqdm(list(self.idf.keys()))}
    
    """
    Transforme un texte en une représentation TF-IDF
    """
    def transform(self, tokens) :
        unique_tokens = list(set(tokens))
        
        tfidf = {}
        for word in unique_tokens:
            tfidf[word] = tokens.count(word)
        
        tfidf = {word : self.idf[word]*tfidf[word] for word in tfidf if word in self.idf}
        return list(tfidf.items())

In [ ]:
print("Testing TFIDF class :")
tfidf_test = TFIDF()
tfidf_test.fit([['video','awesome', 'The'], ['house', 'The']])
tfidf_test.transform(['video','awesome', 'The','The'])

### Word embedding

Récemment, un nouveau type de représentation, appelé word embedding ou word vector, s'est révélé très utile pour la PNL. Dans les plongements de mots, les mots sont représentés comme des vecteurs réels, de faible dimension et denses. Ces vecteurs décrivent les positions des mots dans un nouvel espace de caractéristiques qui conservent les informations syntaxiques et sémantiques. Contrairement à d'autres représentations, word embeddings  souffrent moins de la malédiction de la dimensionnalité et améliorent la capacité du modèle à gérer les mots inconnus et rares dans la formation. Par ailleurs,
en utilisant word embedding, il est possible d'effectuer des opérations arithmétiques et calculer la distance entre les mots. 

Dans ce TP, nous utiliserons des incorporations de mots pour générer une représentation dense du texte, appelée *text embedding*.
Dans ce contexte, le texte peut contenir une phrase ou plusieurs paragraphes.
Le text embedding est calculé comme la moyenne des vecteurs des mots :
\begin{equation}
	e_s = \frac{1}{|s|} \sum_{t \in s} w_t,
\end{equation}
où $|s|$ est la longueur du texte $s$, $w_t$ est word embedding du token $t$ dans $s$ et $e_s$ est text embedding de $s$.

On utilise les vecteurs des mots issus d'un modèle pré-entraîné de *glove.42B.300d_clear.txt* dans le dossier *dataset*.
Dans chaque ligne de ce fichier texte, il y a les tokens et leurs valeurs vectorielles. Les valeurs et les tokens sont séparés par des espaces. Dans ce fichier, la longueur de word embedding est de 300.

In [ ]:
class TextEmbedding:
    def __init__(self):
        word_vec_file = open(word_vec_path)
        
        #Lecture de chaque ligne du fichier pour mettre en forme les vecteurs
        word = word_vec_file.readline()
        words_vectors = {}
        while word != "":
            word = word.split(' ')
            words_vectors[word[0]] = [float(vect) for vect in word[1:]]
            word = word_vec_file.readline()
        self.words_vectors = words_vectors

    """
    Génère les embeddings d'une liste de tokens
    """
    def generate_embedding(self, tokens):
        words_vectors = self.words_vectors
        sum = np.zeros(300)

        N = 0
        for token in tokens:
            if token in list(words_vectors.keys()):
                sum += np.array(words_vectors[token])
                N+=1

        if (N != 0):
            sum = sum/N
            
        return list(sum)


In [ ]:
print("Testing TextEmbedding class : ")
test = TextEmbedding()
print(test.generate_embedding(["are","better"]))

## Pipeline

Le *pipeline* est la séquence d'étapes de prétraitement des données qui transforme les données brutes dans un format qui permet leur analyse.
Le *pipeline* suit les étapes suivantes : 
- Étape 1. Tokenisation et suppression les mots vides dans le résumé et la description de chaque rapport.
- Étape 2. Génération des text embeddings du résumé et de la description pour chaque rapport à partir des textes pré-traités à l'étape précédente.
- Étape 3. Application du stemming au tokens obtenus en fin d'étape 1.
- Étape 4. Apprentissage de l'IDF en utilisant le résumé et la description de tous les rapports dans le ensemble d'entraînement.
- Étape 5. Génération de la représentation TF-IDF du résumé et de la description pour chaque rapport.
- Étape 6. Ajout des nouvelles informations calculées au JSON.


Après l'exécution du pipeline, chaque rapport dans report_index du fichier JSON quatre clés :
     - summary_tfidf : représentation TF-IDF du résumé
     - desc_tfidf : représentation TF-IDF de la description
     - summary_vec : le text embedding du résumé
     - desc_vec : le text embedding  de la description

In [ ]:
"""
Étape 1. Tokenisation et suppression les mots vides dans le résumé et la description de chaque rapport.
"""

#Dossier du pipeline
PIPELINE_PATH = 'data/pipeline'

#Chemins des fichiers contenant les summaries et descriptions filtrés
filtered_summaries_path = os.path.join(PIPELINE_PATH,'filtered_summaries.json')
filtered_descriptions_path = os.path.join(PIPELINE_PATH,'filtered_descriptions.json')

if not os.path.exists(PIPELINE_PATH):
    os.makedirs(PIPELINE_PATH)

#Charger les summaries et descriptions filtrés si déjà existants
if os.path.isfile(filtered_summaries_path) and os.path.isfile(filtered_descriptions_path):
    filtered_summaries = json.load(open(filtered_summaries_path))
    filtered_descriptions = json.load(open(filtered_descriptions_path))

#Sinon créer un fichier JSON avec les summaries et descriptions filtrés
else:
    filtered_summaries = {}
    filtered_descriptions = {}

    for report_id in tqdm.tqdm(list(report_index.keys())):
        filtered_summaries[report_id] = filter_tokens(tokenize_space_punk((report_index[report_id])["summary"]))
        filtered_descriptions[report_id] = filter_tokens(tokenize_space_punk((report_index[report_id])["description"]))
    
    
    json.dump(filtered_summaries, open(filtered_summaries_path,'w'),indent=5)
    json.dump(filtered_descriptions, open(filtered_descriptions_path,'w'),indent= 5)

In [ ]:
"""
Étape 2. Génération des text embeddings du résumé et de la description pour chaque rapport à partir des textes pré-traités à l'étape précédente.
"""

report_index = json.load(open(index_path))
report_index = { int(report_id): data for report_id,data in report_index.items()}
#Chemins des fichiers contenant les embeddings
summaries_embeddings_path = os.path.join(PIPELINE_PATH,'summaries_embeddings.json')
descriptions_embeddings_path = os.path.join(PIPELINE_PATH,'descriptions_embeddings.json')


#Charger les summaries et descriptions filtrés si déjà existants
if os.path.isfile(summaries_embeddings_path) and os.path.isfile(descriptions_embeddings_path):
    summaries_embeddings = json.load(open(summaries_embeddings_path))
    descriptions_embeddings = json.load(open(descriptions_embeddings_path))

else:
    summaries_embeddings = {}
    descriptions_embeddings = {}
    text = TextEmbedding()

    for report_id in tqdm.tqdm(list(filtered_summaries.keys())):
        summaries_embeddings[report_id] = text.generate_embedding(filtered_summaries[report_id])
        descriptions_embeddings[report_id] = text.generate_embedding(filtered_descriptions[report_id])
    
    #Création JSON des embeddings
    json.dump(summaries_embeddings,open(summaries_embeddings_path,'w'))
    json.dump(descriptions_embeddings,open(descriptions_embeddings_path,'w'))



In [ ]:
"""
Étape 3. Application du stemming au tokens obtenus en fin d'étape 1.
"""

#Fichiers des stemmed summaries et stemmed descriptions
stemmed_summaries_path = os.path.join(PIPELINE_PATH,'stemmed_summaries.json')
stemmed_descriptions_path = os.path.join(PIPELINE_PATH,'stemmed_descriptions.json')

#Charger les fichiers si déjà existants
if os.path.isfile(stemmed_summaries_path) and os.path.isfile(stemmed_descriptions_path):
    stemmed_summaries = json.load(open(stemmed_summaries_path))
    stemmed_descriptions = json.load(open(stemmed_descriptions_path))

else:
    stemmed_summaries = {}
    stemmed_descriptions = {}

    for report_id in tqdm.tqdm(list(filtered_summaries.keys())):
        stemmed_summaries[report_id] = [stemmer.stem(token) for token in filtered_summaries[report_id]]
        stemmed_descriptions[report_id] = [stemmer.stem(token) for token in filtered_descriptions[report_id]]
    
    #Dump JSON si les fichiers n'existent pas
    json.dump(stemmed_summaries,open(stemmed_summaries_path,'w'),indent= 5)
    json.dump(stemmed_descriptions,open(stemmed_descriptions_path,'w'),indent= 5)

In [ ]:
"""
Étape 4. Apprentissage de l'IDF en utilisant le résumé et la description de tous les rapports dans le ensemble d'entraînement.
"""
#Création des listes de tokens d'entrainement
training_list_summaries = [stemmed_summaries[report_id] for report_id in list(training_reports_set)]
training_list_descriptions = [stemmed_descriptions[report_id] for report_id in list(training_reports_set)]

#training_reports_set contient les id de tous les rapports de l'ensemble d'entraînement.
training_list = np.unique(training_list_descriptions+training_list_summaries)

#Apprentissage de l'IDF
tfidf = TFIDF()
tfidf.fit(training_list)



In [ ]:
"""
Étape 5. Génération de la représentation TF-IDF du résumé et de la description pour chaque rapport.
"""
#Fichiers des tfidf
tfidf_summaries_path = os.path.join(PIPELINE_PATH,'tfidf_summaries.json')
tfidf_descriptions_path = os.path.join(PIPELINE_PATH,'tfidf_descriptions.json')

#Charger les fichiers si déjà existants
if os.path.isfile(tfidf_summaries_path) and os.path.isfile(tfidf_descriptions_path):
    tfidf_summaries = json.load(open(tfidf_summaries_path))
    tfidf_descriptions = json.load(open(tfidf_descriptions_path))

else:
    tfidf_summaries = {}
    tfidf_descriptions = {}

    for report_id in tqdm.tqdm(list(stemmed_summaries.keys())):
        tfidf_summaries[report_id] = tfidf.transform(stemmed_summaries[report_id])
        tfidf_descriptions[report_id] = tfidf.transform(stemmed_descriptions[report_id])
    
    #Dump JSON si les fichiers n'existent pas
    json.dump(tfidf_summaries,open(tfidf_summaries_path,'w'))
    json.dump(tfidf_descriptions,open(tfidf_descriptions_path,'w'))

In [ ]:
"""
Étape 6. Ajout des nouvelles informations calculées au JSON.
"""
bug_reports_final_path = 'bug_reports_final.json'

#On charge le bug_reports_final.json si déjà existant
if os.path.isfile(bug_reports_final_path):
    report_index = json.load(open(bug_reports_final_path))

else:
    for report_id in tqdm.tqdm(list(report_index.keys())):
        (report_index[report_id])["summary_tfidf"] = tfidf_summaries[str(report_id)]
        (report_index[report_id])["desc_tfidf"] = tfidf_descriptions[str(report_id)]
        (report_index[report_id])["summary_vec"] = summaries_embeddings[str(report_id)]
        (report_index[report_id])["desc_vec"] = descriptions_embeddings[str(report_id)]
    
    #Dump JSON si le bug_report n'est pas présent
    json.dump(report_index,open("bug_reports_final.json","w"))

### Similarité cosinus

En traitement du langage naturel, la similarité cosinus est une fonction de similarité populaire utilisée pour comparer les vecteurs de documents. Cette fonction mesure à quel point la direction de deux vecteurs est différente et ses valeurs sont comprises entre -1 et 1.

La similarité en cosinus est définie comme :
\begin{equation}
    \operatorname{cos(v, v')} = \frac{\sum_{i=1}^{n} v_i  v_i'}{\sqrt{\sum_{i=1}^{n} v_i^2} \sqrt{\sum_{i=1}^{n}v_i'^2}},
\end{equation}
où $v = (v_1, .., v_n)$ et $v' = (v'_1, .., v'_n)$ sont des vecteurs de $n$ dimensions.

In [ ]:
"""
Calcule la similarité cosinus des TF-IDF de deux rapports r1 et r2
"""
def cosine_sim_tf_idf(r1, r2):
    dicor1 = {}
    dicor2 = {}
    
    for word, tfidf in r1:
        dicor1[word] = tfidf
        dicor2[word] = 0
    for word, tfidf in r2:
        dicor2[word] = tfidf
        if dicor1.get(word) is None:
            dicor1[word] = 0
            
    num = 0
    denum1 = 0
    denum2 = 0
    for word in dicor1:
        num += (dicor1[word] * dicor2[word])
        denum1 += (dicor1[word] * dicor1[word])
        denum2 += (dicor2[word] * dicor2[word])
        
    if denum1 == 0 or denum2 == 0 :
        return 0
    else:
        return num / (math.sqrt(denum1)*math.sqrt(denum2))

In [ ]:
print("Testing TF-IDF cosine similarity : ")
report1 = [('A1', 1.4054), ('A2', 1.4054), ('A3', 2.0)]
report2 = [ ("B1", 1.4054), ("A2", 1.4054), ("A3", 2.0), ("A1", 2.8108)]
print(cosine_sim_tf_idf(report1, report2))

In [ ]:
"""
Calcule la similarité cosinus des embeddings de deux rapports
"""
def cosine_sim_embedding(vec1, vec2):
    num = 0
    denum1 = 0
    denum2 = 0
    for i in range(len(vec1)):
        num += (vec1[i] * vec2[i])
        denum1 += (vec1[i] * vec1[i])
        denum2 += (vec2[i] * vec2[i])
    if denum1 == 0 or denum2 == 0 :
        return 0
    else:  
        return num / (math.sqrt(denum1)*math.sqrt(denum2))

## Extraction de features / Feature extraction

Nous formons un modèle de régression logistique pour prédire si une paire de rapports se rapportent au même bug ou non. Les features utilisées pour la classification sont énumérées ci-dessous :

1. Similitude en cosinus de la représentation TF-IDF des résumés des deux rapports.
2. Similitude en cosinus de la représentation TF-IDF des descriptions des deux rapports.
3. Similitude en cosinus des embeddings des résumés des deux rapports.
4. Similitude en cosinus des embeddings des descriptions des deux rapports.
5. Une feature binaire qui est 1.0 lorsque les deux rapports ont le même produit spécifié. Sinon, c'est 0.0.
6. Une feature binaire qui est 1.0 lorsque les deux rapports ont le même composant spécifié. Sinon, c'est 0.0.

In [ ]:
"""
    Extraire les features d'une paire de rapports de bugs (r1, r2).
    rm_ftr_idxs permet de retirer des features lors de l'apprentissage selon les indexes spécifiés plus haut.
    Par exemple, rm_ftr_idxs = [3] permet un apprentissage en utilisant l'ensemble des features excepté la similitude en cosinus des 
"""
def extract_features(r1, r2, rm_ftr_idxs=[]):
    vector = []
    if 1 not in rm_ftr_idxs:
        vector.append(cosine_sim_tf_idf(r1["summary_tfidf"], r2["summary_tfidf"]))
    if 2 not in rm_ftr_idxs:
        vector.append(cosine_sim_tf_idf(r1["desc_tfidf"], r2["desc_tfidf"]))
    if 3 not in rm_ftr_idxs:
        vector.append(cosine_sim_embedding(r1["summary_vec"], r2["summary_vec"]))
    if 4 not in rm_ftr_idxs:
        vector.append(cosine_sim_embedding(r1["desc_vec"], r2["desc_vec"]))
    if 5 not in rm_ftr_idxs:
        if r1["component"] == r2["component"]:
            vector.append(1.0)
        else:
            vector.append(0.0)
    if 6 not in rm_ftr_idxs:
        if r1["product"] == r2["product"]:
            vector.append(1.0)
        else:
            vector.append(0.0)
    return vector

## Entraînement et test du modèle

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# Charger des étiquettes à partir de l'ensemble d'apprentissage
y_train = np.asarray([ y for _, _, y in  training_pairs ])

def train_clf(rm_ftr_idxs=[]):
    # Extraire les features 
    X_train = np.asarray([extract_features(report_index[str(r1)], report_index[str(r2)], rm_ftr_idxs) for r1, r2, _ in  training_pairs ])
    return LogisticRegression(random_state=0).fit(X_train, y_train)

In [ ]:
def compute_acc(classifier, X):
    # Compute accuracy
    y = np.asarray([ y for _, _, y in  validation_pairs ])
    return classifier.score(X, y)

In [ ]:
#Mise en forme des données de validation
X_test_1 = np.asarray([extract_features(report_index[str(r1)], report_index[str(r2)]) for r1, r2, _ in validation_pairs])
X_test_2 = np.asarray([extract_features(report_index[str(r1)], report_index[str(r2)],[1,2]) for r1, r2, _ in validation_pairs])
X_test_3 = np.asarray([extract_features(report_index[str(r1)], report_index[str(r2)],[3,4]) for r1, r2, _ in validation_pairs])
X_test_4 = np.asarray([extract_features(report_index[str(r1)], report_index[str(r2)],[5]) for r1, r2, _ in validation_pairs])
X_test_5 = np.asarray([extract_features(report_index[str(r1)], report_index[str(r2)],[6]) for r1, r2, _ in validation_pairs])

#Affichage des précisions
print("Classifier Accuracy According to the Used Features:\n")
print("1. Classifier with all features", f"{compute_acc(train_clf(), X_test_1)*100:.2f} %")
print("2. Classifier without TF-IDF cosine similarity:", f"{compute_acc(train_clf([1,2]), X_test_2)*100:.2f} %")
print("3. Classifier without embedding cosine similarity:", f"{compute_acc(train_clf([3,4]), X_test_3)*100:.2f} %")
print("4. Classifier without component comparison:", f"{compute_acc(train_clf([5]), X_test_4)*100:.2f} %")
print("5. Classifier without product comparison:", f"{compute_acc(train_clf([6]), X_test_5)*100:.2f} %")

## Conclusion

Le classificateur avec toutes les caractéristiques offre la meilleure précision (94,75 %), mais la différence de précision entre ce classificateur et ceux sans caractéristiques spécifiques (similitude cosinus TF-IDF, similitude cosinus d'embeddings, comparaison de composants ou comparaison de produits) est relativement faible. L'utilisation du NLP et plus précisément de la méthode bag-of-words permet de détecter les doublons des rapports de bugs dans une très grande majorité des cas. Un compromis entre précision du détecteur et coût de calcul en omettant certaines caractéristiques peut être fait selon les préférences pour le BTS.